<a href="https://colab.research.google.com/github/emakhankov/stepik_samsung_computer_vision/blob/master/stepik_samsung_computer_vision_version_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# By Evgeny Makhankov

In [0]:
# Гипотеза что сеть разпознающее флюораграфические снимки, возможно будет хорошо справляться с обнаружением грязной посуды

# Примеры сети
# https://www.kaggle.com/curiousprogrammer/chest-x-ray-image-classification-tf-hub-resnet50
# https://www.kaggle.com/curiousprogrammer/chest-x-ray-keras-vgg19-transfer-learning

In [0]:
# Инсталлируем tensorflow, т.к. в нем есть предобученная моделль ResNet50V2
pip install tensorflow==2.0

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os # Работа с файлами
import gc # Сбоока мусора, т.к. без принудительной сборки колаб вылетал

#File Operation libraries
import glob
from pathlib import Path

#Visualisation Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_confusion_matrix

#Image Transformation Libraries
import cv2
from imgaug import augmenters as iaa # Пакет по аугментации

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle competitions download -c platesv2

In [0]:
!unzip plates.zip

In [0]:
base_dir = "plates/"
train_dir = base_dir+'train/'
test_dir = base_dir+'test/'
val_dir = base_dir+'val/'

In [0]:
#Разбивка данных на train и val datasets
#Изначально для сети ResNet50V2 хороший результат (0.87) был получен при i % 6, потом доучивая сеть с увеличением train и даже с полной отказом от val - выше результата не дало.
#Последная сеть ResNet50V2 + Inception уже валидировалась по аугментированной обучающей выборке

def get_df(path,shuffle = True,validation = False):
    lst = []
    cleaned_dir = Path(path + "cleaned")
    dirty_dir = Path(path + "dirty")
    cleaned_data = cleaned_dir.glob("*.jpg")
    dirty_data = dirty_dir.glob("*.jpg")
    
    i = 0;
    for fname in cleaned_data:
      i += 1;
      #if (validation == True) and (i % 12 == 0) or (validation == False) and (i % 12 != 0) :
      if True: #Валидировать будем по аугментированным данным
          lst.append((fname, 0))
    i = 0;
    for fname in dirty_data:
      i += 1;
      if True: 
      #if (validation == True) and (i % 12 == 0) or (validation == False) and (i % 12 != 0) :
        lst.append((fname, 1))
    df = pd.DataFrame(lst, columns=['Image', 'Label'], index=None)
    s = np.arange(df.shape[0])
    if shuffle:
      np.random.shuffle(s)
    df = df.iloc[s,:].reset_index(drop=True)
    return df

In [0]:
df_train = get_df(train_dir)
df_train = pd.concat([df_train,df_train,df_train])  # Увеличим обучающую выборку в 3 раза, потом добросить проблемные элементы, чтобы они в общей массе были не столь значимы
df_val = get_df(train_dir,False,True)

In [0]:
# ResNet50V2 + Inception добавляление 3-х картинк - металлическая кастюля с затемненным дном. (часто определяется как грязная, причем глазами тяжело было понять это прилипшее кофе или тень)
lst = []
lst.append((Path("plates/test/0370.jpg"), 0))
lst.append((Path("plates/test/0178.jpg"), 0))
lst.append((Path("plates/test/0701.jpg"), 0))
df = pd.DataFrame(lst, columns=['Image', 'Label'], index=None)
df = df.reset_index(drop=True)
df_train = pd.concat([df_train,df])